In [ ]:
# Twitter Authentication

import tweepy
from twitter_authentication import API_KEY, API_SECRET, ACCESS_TOKEN, ACCESS_TOKEN_SECRET

oauth1_user_handler = tweepy.OAuth1UserHandler(
    API_KEY, API_SECRET,
    callback="oob"
)

print(oauth1_user_handler.get_authorization_url())

verifier = input("Input PIN: ")
access_token, access_token_secret = oauth1_user_handler.get_access_token(
    verifier
)
# print(access_token)
# print(access_token_secret)

In [ ]:
import tweepy
from twitter_authentication import API_KEY, API_SECRET, ACCESS_TOKEN, ACCESS_TOKEN_SECRET

# API
auth = tweepy.OAuthHandler(API_KEY, API_SECRET)
auth.set_access_token(ACCESS_TOKEN, ACCESS_TOKEN_SECRET)
api = tweepy.API(auth)

# Client
client = tweepy.Client(
    consumer_key = API_KEY,
    consumer_secret = API_SECRET,
    access_token = ACCESS_TOKEN,
    access_token_secret = ACCESS_TOKEN_SECRET,
)

In [ ]:
# Basic Tweet Test

media = api.media_upload("Media/F-35 LOGO.png")
# print(media)

client.create_tweet(text='t4', media_ids=[media.media_id])

In [ ]:
# Reddit authentication

import praw
from reddit_authentication import CLIENT_ID, SECRET_KEY, USER_AGENT

reddit = praw.Reddit(
    client_id = CLIENT_ID,
    client_secret = SECRET_KEY,
    user_agent = USER_AGENT,
)

In [ ]:
# SQLITE

import sqlite3
con = sqlite3.connect("bot_db.db")
cur = con.cursor()
cur.execute('''
                CREATE TABLE IF NOT EXISTS reddit_post (
                    post_id     TEXT PRIMARY KEY,
                    title       TEXT NOT NULL,
                    flair       TEXT,
                    post_url    TEXT NOT NULL,
                    author      TEXT NOT NULL,
                    post_date   TEXT NOT NULL,
                    scrape_date TEXT NOT NULL,
                    tweet_url   TEXT,
                    tweet_date  TEXT
            );''')

def insert_into_table(db: str, table_name: str, data: list):
    con = sqlite3.connect(db)
    cur = con.cursor()
    
    sql = f"INSERT OR IGNORE INTO {table_name} VALUES ({','.join(['?' for _ in data[0]])})"
    cur.executemany(sql, data)
    
    con.commit()
    con.close()

def update_table(db: str, table_name: str, set_values_dict: dict, where_clause: str):
    con = sqlite3.connect(db)
    cur = con.cursor()

    set_clause = ', '.join([f"{col} = ?" for col in set_values_dict.keys()])
    sql = f"UPDATE {table_name} SET {set_clause} WHERE {where_clause}"
    cur.execute(sql, tuple(set_values_dict.values()))

    con.commit()
    con.close()

In [ ]:
# Hardcoded cell to test specific posts

from datetime import datetime, timezone, timedelta

post_data = []
submission = reddit.submission(url='https://www.reddit.com/r/NonCredibleDefense/comments/11rhzex/my_recreation_of_the_events_in_the_black_sea_today/')

if not submission.stickied and not submission.is_self:
    # print(f'Post: {submission.id, submission.name, submission.title, submission.selftext}')
    # print(f'Flair: {submission.link_flair_text}')
    # print(f'Status: {submission.over_18, submission.spoiler, submission.stickied, submission.created_utc}')
    # print(f'Links: reddit.com{submission.permalink}, {submission.url}')
    # print(f'Author: {submission.author.name}')
    # print(submission.score)
    
    # print(submission.is_video)
    # print(submission.media)
    # print(submission.url)

    post_date = datetime.fromtimestamp(submission.created_utc) + timedelta(hours = 3) # Adds 3 hours because local time is GMT-3. Review this.
    datetime_now = datetime.now(timezone.utc)
    scrape_date = datetime_now.strftime("%Y-%m-%d %H:%M:%S")

    post_data_row = (
            submission.id,                          # post_id     TEXT PRIMARY KEY,
            submission.title,                       # title       TEXT NOT NULL,
            submission.link_flair_text,             # flair       TEXT,
            'reddit.com' + submission.permalink,    # post_url    TEXT NOT NULL,
            submission.author.name,                 # author      TEXT NOT NULL,
            submission.created_utc,                 # post_date   TEXT NOT NULL,
            scrape_date,                            # scrape_date TEXT NOT NULL,
            None,
            None
    )
    post_data.append(post_data_row)

# insert_into_table('bot_db.db', 'reddit_post', post_data)

In [ ]:
from datetime import datetime, timezone, timedelta

post_data = []

for submission in reddit.subreddit("NonCredibleDefense").top(limit = 5, time_filter = "day"):
    if not submission.stickied and not submission.is_self:
        # print(f'Post: {submission.id, submission.name, submission.title, submission.selftext}')
        # print(f'Flair: {submission.link_flair_text}')
        # print(f'Status: {submission.over_18, submission.spoiler, submission.stickied, submission.created_utc}')
        # print(f'Links: reddit.com{submission.permalink}, {submission.url}')
        # print(f'Author: {submission.author.name}')
        # print(submission.score)

        post_date = datetime.fromtimestamp(submission.created_utc) + timedelta(hours = 3) # Adds 3 hours because local time is GMT-3. Review this.
        datetime_now = datetime.now(timezone.utc)
        scrape_date = datetime_now.strftime("%Y-%m-%d %H:%M:%S")

        post_data_row = (
                submission.id,                          # post_id     TEXT PRIMARY KEY,
                submission.title,                       # title       TEXT NOT NULL,
                submission.link_flair_text,             # flair       TEXT,
                'https://www.reddit.com' + submission.permalink,    # post_url    TEXT NOT NULL,
                submission.author.name,                 # author      TEXT NOT NULL,
                submission.created_utc,                 # post_date   TEXT NOT NULL,
                scrape_date,                            # scrape_date TEXT NOT NULL,
                None,
                None
        )
        post_data.append(post_data_row)
        print(post_data_row)

        # if hasattr(submission, 'gallery_data'):
        #     for item in sorted(submission.gallery_data['items'], key=lambda x: x['id']):
        #         media_id = item['media_id']
        #         media_metadata = submission.media_metadata[media_id]
        #         print(media_metadata['s']['u'])
        #         media_data_row = (
        #                 submission.id,                          # post_id     TEXT NOT NULL,
        #                 media_metadata['s']['u'],               # url         TEXT NOT NULL,
        #         )
        #         # print(media_data_row)
        #         media_data.append(media_data_row)
        # else:
        #     media_data_row = (
        #             submission.id,                          # post_id     TEXT NOT NULL,
        #             submission.url,                         # url         TEXT NOT NULL,
        #     )
        #     media_data.append(media_data_row)
        # print(media_data)

insert_into_table('bot_db.db', 'reddit_post', post_data)

# tweet_url   TEXT,
# tweet_date  TEXT

        # title = submission.title
        # flair = submission.link_flair_text
        # if flair:
        #     flair = flair.translate(str.maketrans('&', 'n', '!/ '))
        # submission_url = 'reddit.com' + submission.permalink
        # media_url = submission.url
        # author = submission.author.name
        # user_profile_url = 'reddit.com/u/' + user

In [ ]:
# url = 'https://www.reddit.com/r/dogelore/comments/jbu197/he_chose_poorly/'
# submission = reddit.submission(url=url)

url_list = [
    'https://www.reddit.com/r/dogelore/comments/jbu197/he_chose_poorly/', 
    'https://www.reddit.com/r/NonCredibleDefense/comments/t0o3lc/godspeed_ghost_of_kyiv_may_the_razgriz_be_with_you/', 
    'https://www.reddit.com/r/NonCredibleDefense/comments/11bimqt/pov_its_20220225_second_day_kyiv_time_you_are/'
]

for item in url_list:
    submission = reddit.submission(url=item)
    print(hasattr(submission, 'gallery_data'))
    # for item in sorted(submission.gallery_data['items'], key=lambda x: x['id']):
    #     print(f'Post: {submission.id, submission.name, submission.title, submission.selftext}')
    #     print(f'Flair: {submission.link_flair_text}')
    #     print(f'Status: {submission.over_18, submission.spoiler, submission.stickied, submission.created_utc}')
    #     print(f'Links: reddit.com{submission.permalink}, {submission.url}')
    #     print(f'Author: {submission.author.name}')
    #     print()

# GALLERY CASE

# url = 'https://www.reddit.com/r/NonCredibleDefense/comments/11bimqt/pov_its_20220225_second_day_kyiv_time_you_are/'
url = 'https://www.reddit.com/r/dogelore/comments/jbu197/he_chose_poorly/'
# url = 'https://www.reddit.com/r/NonCredibleDefense/comments/11pkssc/any_day_now/'
submission = reddit.submission(url=url)
# print(len(sorted(submission.gallery_data['items'], key=lambda x: x['id'])))
for item in sorted(submission.gallery_data['items'], key=lambda x: x['id']):
    media_id = item['media_id']
    media_metadata = submission.media_metadata[media_id]
    print(media_metadata['s']['u'])
    print(type(media_metadata['s']['u']))

In [ ]:
import os
import shutil

MEDIA_FOLDER_PATH = './temp_media/'

def clear_media_folder():
    folder_path = MEDIA_FOLDER_PATH
    try:
        for filename in os.listdir(folder_path):
            file_path = os.path.join(folder_path, filename)
            if os.path.isfile(file_path):
                os.remove(file_path)
            else:
                shutil.rmtree(file_path)
    except:
        print("An exception occurred when trying to clear the media folder.")

# clear_media_folder()

In [ ]:
from RedDownloader import RedDownloader

url = 'https://www.reddit.com/r/NonCredibleDefense/comments/11sbyta/things_have_really_changed_in_the_past_20_years/'

def download_media_from_reddit(url: str):
    clear_media_folder()
    file = RedDownloader.Download(url, quality=1080, destination=MEDIA_FOLDER_PATH)
    # print(file.GetMediaType())

# download_media_from_reddit(url)

In [ ]:
import os

MEDIA_FOLDER_PATH = './temp_media/'

def upload_media_to_twitter():
    media_ids = []
    for path, subdirs, files in os.walk(MEDIA_FOLDER_PATH):
        for name in files:
            file_path = os.path.join(path, name)
            try:
                media = api.media_upload(file_path)
                media_ids.append(media.media_id)
                # print(media.media_id)
            except:
                print(f"An exception occurred when trying to upload this media file to Twitter: {file_path}.")
    
    return media_ids
    

# def upload_media():
#     folder_path = MEDIA_FOLDER_PATH
#     for filename in os.listdir(folder_path):
#         file_path = os.path.join(folder_path, filename)
#         print(file_path)

# upload_media()

In [ ]:
import textwrap
from time import sleep

TWEET_CHARACTER_LIMIT = 280
TWEET_URL_LENGTH = 23
TWEET_STRUCTURE_LENGTH = 8

def create_tweet():
    # Retrieve one row from reddit_post
    query = '''
                SELECT *
                FROM reddit_post
                WHERE tweet_url IS NULL AND tweet_date IS NULL
                LIMIT 1
            '''
    res = cur.execute(query).fetchall()
    post_id, title, flair, post_url, author, post_date, scrape_date, tweet_url, tweet_date = res[0]
    print(res)

    download_media_from_reddit(post_url)

    # flair = 'test1'
    # title = 'title test'
    # post_url = 'https://url_test.com'

    media_ids = upload_media_to_twitter()

    # Get rid of spaces and other characters in the flair and turn it into a hashtag
    if flair:
        flair = flair.translate(str.maketrans('&', 'n', '!/ '))

    width = TWEET_CHARACTER_LIMIT - TWEET_URL_LENGTH - len(flair) - TWEET_STRUCTURE_LENGTH
    
    # Wrap the title in case it's too long
    title_wrapped = textwrap.shorten(title, width, placeholder='...')
    
    tweet_text = f"""
    {title_wrapped} #{flair}\n\nOP: {post_url}
    """
    tweet_text = tweet_text[1:-1]

    # Tweet creation
    if len(media_ids) <= 4:     # If 4 media items or less, it creates the tweet
        response = client.create_tweet(text=tweet_text, media_ids=media_ids)
    else:                       # If more than 4 media items, it posts the first 4 images and then creates a thread replying the last tweet.
        response = client.create_tweet(text=tweet_text, media_ids=media_ids[:4])
        previous_tweet_id = response[0]['id']
        media_ids = media_ids[4:]
        while len(media_ids) > 0:
            sleep(5)
            response = client.create_tweet(in_reply_to_tweet_id=previous_tweet_id, media_ids=media_ids[:4])
            previous_tweet_id = response[0]['id']
            media_ids = media_ids[4:]

create_tweet()




# title = submission.title
# flair = submission.link_flair_text

# submission_url = 'reddit.com' + submission.permalink
# media_url = submission.url
# author = submission.author.name
# user_profile_url = 'reddit.com/u/' + user

In [ ]:
import textwrap

TWEET_CHARACTER_LIMIT = 280
TWEET_URL_LENGTH = 23
TWEET_STRUCTURE_LENGTH = 8


width = TWEET_CHARACTER_LIMIT - TWEET_URL_LENGTH - len(hashtag) - TWEET_STRUCTURE_LENGTH
title = title
title_wrapped = textwrap.shorten(title, width, placeholder='...')
tweet_text = f"""
{title_wrapped} #{hashtag}

OP: {submission_url}
"""
tweet_text = tweet_text[1:-1]
print(tweet_text)
print()
print(f' Text length: {len(tweet_text[1:-1])}')
print(f'Tweet length: {len(tweet_text[1:-1]) - len(submission_url) + TWEET_URL_LENGTH}')
print(width)
print(width + TWEET_URL_LENGTH + len(hashtag) + 8)


In [ ]:
# https://praw.readthedocs.io/en/stable/getting_started/quick_start.html
# https://github.com/reddit-archive/reddit/wiki/API#rules
# https://docs.python.org/3/library/sqlite3.html

# VIDEO CASES
# url = 'https://www.reddit.com/r/NonCredibleDefense/comments/11b2sxa/i_think_ukrainian_minister_of_defense_just/'
# url = 'https://www.reddit.com/r/NonCredibleDefense/comments/t0o3lc/godspeed_ghost_of_kyiv_may_the_razgriz_be_with_you/'

# GALLERY CASES
# url = 'https://www.reddit.com/r/dogelore/comments/jbu197/he_chose_poorly/'
# url = 'https://www.reddit.com/r/announcements/comments/hrrh23/now_you_can_make_posts_with_multiple_images/'
# url = 'https://www.reddit.com/r/hvacadvice/comments/x7cep9/black_soot_at_electric_box/'
# url = 'https://www.reddit.com/r/ProCreate/comments/10wdd7c/gallery_post_description_in_the_comment/'

# SPECIAL GALLERY CASE
# url = 'https://www.reddit.com/r/NonCredibleDefense/comments/11bimqt/pov_its_20220225_second_day_kyiv_time_you_are/'

In [ ]:
### SQLite tools ###

for row in cur.execute("SELECT * FROM reddit_post"):
    print(row)

# for row in cur.execute("SELECT rowid, * FROM post_media"):
#     print(row)

# cur.execute(query, (post_id,))
# for row in cur.execute(query, (post_id,)):
#     print(row)

# cur.execute("PRAGMA foreign_keys = ON")
# con.commit()
# cur.execute("DELETE FROM reddit_post WHERE post_id = 'jbu197'")
# con.commit()

In [ ]:
### DateTime and Unix tools ###

from datetime import datetime, timezone, timedelta

# UTC
print(datetime.now(timezone.utc).strftime("%Y-%m-%d %H:%M:%S"))
# Unix from UTC
print(int(datetime.now(timezone.utc).timestamp()))

# res = cur.execute('SELECT datetime("1677401305.0")')
# res = cur.execute('SELECT datetime()')
res = cur.execute('SELECT DATETIME(1677363730.0, "unixepoch")')
print(res.fetchall()[0][0])

# SQLITE    2023-02-26 21:05:22
# PYTHON    2023-02-26 18:05:22

from datetime import datetime, timezone, timedelta


unix_time = 1677363730.0
datetime_obj = datetime.fromtimestamp(unix_time) #+ timedelta(hours = 3)


print(datetime_obj)

dt = datetime.now(timezone.utc)
print(dt.strftime("%Y-%m-%d %H:%M:%S"))